## EMA Split Margin Feature by Greyhound

Here we'll determine the EMA Split Margin for each greyhound

----

Import libraries, packages, and greyhound data

In [12]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
import os
import decouple
import sys
config = decouple.AutoConfig(' ')
os.chdir(config('ROOT_DIRECTORY'))
sys.path.insert(0, '')

from scipy.stats import zscore
from multielo import MultiElo, Player, Tracker
from multielo.multielo import defaults

# Read in data
df_raw = pd.read_csv('./data/clean/dog_results.csv')

display(df_raw)

,FasttrackDogId,Place,DogName,Box,Rug,Weight,StartPrice,Margin1,Margin2,PIR,...,FasttrackRaceId,TrainerId,TrainerName,Distance,RaceGrade,Track,RaceNum,TrackDist,RaceDate,FieldSize
0,157500927,1,RAINE ALLEN,1,1,27.4,2.4,2.30,NaN,Q/111,...,335811282,7683,C GRENFELL,500.0,Restricted Win,Bendigo,1.0,Bendigo500,2018-07-01,6
1,1820620018,2,SURF A LOT,2,2,32.8,6.3,2.30,2.30,M/332,...,335811282,137227,C TYLEY,500.0,Restricted Win,Bendigo,1.0,Bendigo500,2018-07-01,6
2,1950680026,3,PINGIN' BEE,6,6,25.5,9.3,3.84,1.54,S/443,...,335811282,132763,P DAPIRAN,500.0,Restricted Win,Bendigo,1.0,Bendigo500,2018-07-01,6
3,1524380048,4,LUCAS THE GREAT,7,7,32.2,9.1,5.27,1.43,M/655,...,335811282,116605,E HAMILTON,500.0,Restricted Win,Bendigo,1.0,Bendigo500,2018-07-01,6
4,124225458,5,QUAVO,4,4,28.9,3.4,5.56,0.29,M/766,...,335811282,132763,P DAPIRAN,500.0,Restricted Win,Bendigo,1.0,Bendigo500,2018-07-01,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
782997,491585906,3,GLORIOUS GUNN,8,8,27.1,3.8,3.75,2.43,6644,...,745616339,87891,G HORNE,520.0,Grade 5,Cannington,12.0,Cannington520,2021-12-31,7
782998,485659451,4,WOOD FIRE,3,3,32.1,4.1,3.75,0.14,3233,...,745616339,68549,C HALSE,520.0,Grade 5,Cannington,12.0,Cannington520,2021-12-31,7
782999,528381655,5,TRENDING QUARTER,6,6,31.8,16.2,5.25,1.43,4566,...,745616339,83581,J DAILLY,520.0,Grade 5,Cannington,12.0,Cannington520,2021-12-31,7
783000,537992387,6,ELITE WEAPON,1,1,26.7,2.9,5.25,0.00,1455,...,745616339,293372,S WILLIAMS,520.0,Grade 5,Cannington,12.0,Cannington520,2021-12-31,7


Let's count the number of (Track, Dist) with NaN value percentages. We'll only take (Track, Dist) with at least 80% of rows containing a recorded SplitMargin.

In [13]:
# Copy raw dataframe
df = df_raw.copy()

# Group by (Track, Distance) and count NaN values
df = df.groupby('TrackDist', as_index=False).agg(NumberOfVals = ('SplitMargin', 'count'),
                                                 TotalNumber = ('FasttrackDogId', 'count'))

# Create percentage column
df['Percentage'] = round(100*df['NumberOfVals']/df['TotalNumber'], 2)

# Take a list of all (Track, Dist) that have at least 70% of SplitMargin's recorded
df = df[df['Percentage'] >= 70]
trackdist_list = df.TrackDist.to_list()

Here we take only (Track, Dist) values that have at least 80% of SplitMargin values recorded, remove outlier values (incorrectly entered values), then break down SplitMargin performance into 10 quantiles by (Track, Distance). We also replace NaN values with the respective Speed quantile by (Track, Dist).

In [14]:
# Copy raw dataframe
df = df_raw.copy()

# Remove (Track, Dist) rows that don't contain enough SplitMargin data
df = df[df.TrackDist.isin(trackdist_list)]

# Invert SplitMargin (larger is faster)
df['SplitMargin'] = 1/df['SplitMargin']

# Calculate quantiles for speed by (Track, dist)
df['Speed'] = df['Distance']/df['RunTime']
df['SpeedQuantile'] = df.groupby('TrackDist')['Speed'].transform(lambda x: pd.qcut(x, 10, labels=False)+1)

# Calculate quantiles for SplitMargin by (Track, Dist)
df_temp = df.copy()[['FasttrackDogId', 'FasttrackRaceId', 'TrackDist', 'SplitMargin']]
df_temp = df_temp[~df_temp.SplitMargin.isna()]
df_temp['SplitMarginQuantile'] = df_temp.groupby('TrackDist')['SplitMargin'].transform(lambda x: pd.qcut(x, 10, labels=False)+1)

# Merge to original dataframe
df = df.merge(df_temp.drop('SplitMargin', axis=1), on=['FasttrackDogId', 'FasttrackRaceId', 'TrackDist'], how='left')

# Fill SplitMarginQuantile NaN values with SpeedQuantile (correlated)
df['SplitMarginQuantile'] = df['SplitMarginQuantile'].fillna(df['SpeedQuantile'])

display(df)

,FasttrackDogId,Place,DogName,Box,Rug,Weight,StartPrice,Margin1,Margin2,PIR,...,Distance,RaceGrade,Track,RaceNum,TrackDist,RaceDate,FieldSize,Speed,SpeedQuantile,SplitMarginQuantile
0,157500927,1,RAINE ALLEN,1,1,27.4,2.4,2.30,NaN,Q/111,...,500.0,Restricted Win,Bendigo,1.0,Bendigo500,2018-07-01,6,17.445918,7.0,8.0
1,1820620018,2,SURF A LOT,2,2,32.8,6.3,2.30,2.30,M/332,...,500.0,Restricted Win,Bendigo,1.0,Bendigo500,2018-07-01,6,17.349063,6.0,2.0
2,1950680026,3,PINGIN' BEE,6,6,25.5,9.3,3.84,1.54,S/443,...,500.0,Restricted Win,Bendigo,1.0,Bendigo500,2018-07-01,6,17.283097,4.0,1.0
3,1524380048,4,LUCAS THE GREAT,7,7,32.2,9.1,5.27,1.43,M/655,...,500.0,Restricted Win,Bendigo,1.0,Bendigo500,2018-07-01,6,17.223562,3.0,1.0
4,124225458,5,QUAVO,4,4,28.9,3.4,5.56,0.29,M/766,...,500.0,Restricted Win,Bendigo,1.0,Bendigo500,2018-07-01,6,17.211704,3.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
661793,491585906,3,GLORIOUS GUNN,8,8,27.1,3.8,3.75,2.43,6644,...,520.0,Grade 5,Cannington,12.0,Cannington520,2021-12-31,7,17.077176,6.0,3.0
661794,485659451,4,WOOD FIRE,3,3,32.1,4.1,3.75,0.14,3233,...,520.0,Grade 5,Cannington,12.0,Cannington520,2021-12-31,7,17.071569,6.0,7.0
661795,528381655,5,TRENDING QUARTER,6,6,31.8,16.2,5.25,1.43,4566,...,520.0,Grade 5,Cannington,12.0,Cannington520,2021-12-31,7,17.015707,5.0,5.0
661796,537992387,6,ELITE WEAPON,1,1,26.7,2.9,5.25,0.00,1455,...,520.0,Grade 5,Cannington,12.0,Cannington520,2021-12-31,7,17.015707,5.0,9.0


We will now calculate the exponential moving average SplitMargin for each greyhound

In [15]:
# Take EMA of SplitMarginQuantile and shift by one (prevent data leakage)
df = df.sort_values(by=['RaceDate', 'FasttrackDogId'])
alpha_ = 0.2
df['SplitMarginQuantileEMA'] = df.groupby('FasttrackDogId')['SplitMarginQuantile'].transform(lambda x: x.ewm(alpha=alpha_).mean().shift(1))

# Take only columns of interest
df = df[['FasttrackDogId', 'FasttrackRaceId', 'TrackDist', 'SplitMarginQuantileEMA']]

display(df)

,FasttrackDogId,FasttrackRaceId,TrackDist,SplitMarginQuantileEMA
48,-750768,335811289,Bendigo425,NaN
351,109032131,334311905,Mount Gambier400,NaN
329,109032145,334309959,Albion Park331,NaN
364,109032152,334311907,Mount Gambier512,NaN
37,109032166,335811287,Bendigo425,NaN
...,...,...,...,...
661300,580794426,747018056,Geelong400,8.776335
661479,581314812,747048622,Bendigo425,4.948509
661558,587948053,745289885,Ipswich288,8.555556
661555,591659457,745289884,Ipswich288,2.000000


Save to ./data/features as a .csv

In [16]:
df.to_csv('./data/features/ema-split-margin-by-greyhound.csv', index=False)